In [1]:
!pip install selenium

     ---------------------------------------- 6.9/6.9 MB 4.2 MB/s eta 0:00:00
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import re

import pandas as pd

In [2]:
driver = webdriver.Chrome(executable_path = "D:/heejun/dev/chromedriver.exe")

C:\Users\희준\AppData\Local\Temp\ipykernel_10736\2300105881.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "D:/heejun/dev/chromedriver.exe")


In [47]:
title_list = []
speaker_list = []
for i in range(10):
    url = 'https://www.ted.com/talks?page={}&sort=popular'.format(i)
    driver.get(url)
    driver.implicitly_wait(time_to_wait=5)
    soup = BeautifulSoup(driver.page_source, 'html')
    soup= soup.find()
    script_list = soup.findAll(class_="h9 m5 f-w:700") # 타겟이 될 class
    script_list2 = soup.findAll(class_="h12 talk-link__speaker")
    for j in range(len(script_list)):
        title_list.append(script_list[j].text[2:-2])
    for k in range(len(script_list2)):
        speaker_list.append(script_list2[k].text)

In [48]:
def make_link_list(title_list, speaker_list):        
    link_list = []
    for i in range(len(title_list)):
        link = speaker_list[i]+'_'+title_list[i]
        link = link.replace(" ","_")
        link = link.replace("'","_")
        link = link.replace(".","")
        link = link.replace(",","")
        link = link.replace("_—","")
        link = link.replace("?","")
        link = link.replace(":","")
        link = link.replace("(","")
        link = link.replace(")","")
        link = link.replace("é","e")
        link_list.append(link)
        
    return link_list

In [49]:
link_list = make_link_list(title_list, speaker_list)

In [50]:
# 긁어온 제목별로 스크립트 크롤링
script_corpus=[]
for link in link_list:
    url = 'https://www.ted.com/talks/{}/transcript'.format(link)
    driver.get(url)
    driver.implicitly_wait(time_to_wait=5)
    soup = BeautifulSoup(driver.page_source, 'html')
    script_list = soup.findAll(class_="inline cursor-pointer hover:bg-red-300 css-82uonn") # 타겟이 될 class
    script = []
    for i in range(len(script_list)):
        script.append(script_list[i].text)
    script_all = ''.join(script)
    script_corpus.append(script_all)

In [51]:
# 중간 중간에 몇 개는 출력이 안된다. 이거 해결해야함.
for i in range(len(script_corpus)):
    print(len(script_corpus[i]),link_list[i])

17575 Sir_Ken_Robinson_Do_schools_kill_creativity
20055 Amy_Cuddy_Your_body_language_may_shape_who_you_are
12672 Tim_Urban_Inside_the_mind_of_a_master_procrastinator
16779 Simon_Sinek_How_great_leaders_inspire_action
16924 Brene_Brown_The_power_of_vulnerability
9108 Julian_Treasure_How_to_speak_so_that_people_want_to_listen
8914 Sam_Berns_My_philosophy_for_a_happy_life
9644 Robert_Waldinger_What_makes_a_good_life_Lessons_from_the_longest_study_on_happiness
6727 Bill_Gates_The_next_outbreak_We_re_not_ready
13155 Mary_Roach_10_things_you_didn_t_know_about_orgasm
8931 Cameron_Russell_Looks_aren_t_everything_Believe_me_I_m_a_model
11065 Graham_Shaw_Why_people_believe_they_can_t_draw
3764 Tom_Thum_The_orchestra_in_my_mouth
15980 Chimamanda_Ngozi_Adichie_The_danger_of_a_single_story
18006 Susan_Cain_The_power_of_introverts
18298 Pamela_Meyer_How_to_spot_a_liar
8764 Apollo_Robbins_The_art_of_misdirection
16825 Mel_Robbins_How_to_stop_screwing_yourself_over
32983 Elon_Musk_The_future_we_re_bui

In [59]:
pd_dict = {"title":title_list,
          "speaker":speaker_list,
          "script":script_corpus}
script_df = pd.DataFrame(pd_dict)

In [61]:
script_df.to_csv("script_df.csv")

In [ ]:
#텍스트 전처리
def clean_text(inputString):
  text_rmv = re.sub('[\n]', "", inputString)
  return text_rmv